# Testing

In [160]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib
import os

from ScreenTime import ScreenTime

## get data

In [161]:

screen = ScreenTime(757680000, 758700000) # jan 4
#gets (appname, (year, month, hour, minute, second, dayOfWeek))
data = screen.get_screen_time_data() # jan 4 - January 16 2025 6:16:40 PM

# appname to index
apps = screen.get_app_names()

In [63]:
# print(data)
# print(apps)

In [162]:
def convertNp(data, apps):
    len_data = len(data)

    X_test = np.zeros((len_data, 3))
    Y_test = np.zeros((len_data, 20))

    for i in range(len_data):
        x = data[i][1]
        y = data[i][0]

        X_test[i][0] = x[1]/12 # divide to squish it 
        X_test[i][1] = x[2]/24
        X_test[i][2] = x[5]/6

        if y not in apps:
            ind = apps.index("off")
            Y_test[i][ind] = 1
        else:
            ind = apps.index(y)
            Y_test[i][ind] = 1
    return X_test, Y_test

In [163]:

X_train, Y_train = convertNp(data, apps)
# for i in range(20, 50):
#     print(Y_train[i])

## model

In [173]:
model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(3,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(20, activation='softmax')
    ])
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.fit(X_train, Y_train, epochs=10)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6168 - loss: 2.7523  
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7415 - loss: 1.4361
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7333 - loss: 0.9771
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 986us/step - accuracy: 0.7741 - loss: 0.7921
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - accuracy: 0.7693 - loss: 0.7487
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - accuracy: 0.7506 - loss: 0.7194
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7545 - loss: 0.7023
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - accuracy: 0.7508 - loss: 0.6987
Epoch 9/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step - accuracy: 0.7334 - loss: 0.7487
Epoch 10/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step - accuracy: 0.7599 - loss: 0.6523


In [174]:
testing_screen = ScreenTime(758700000, 758990000)
testing_data_x = testing_screen.get_screen_time_data() # jan 16 to jan 19
testing_data_y = testing_screen.get_app_names()

In [175]:
testing_x, testing_y = convertNp(testing_data_x, testing_data_y)

test_lose, test_acc = model.evaluate(testing_x, testing_y)
print("test accuracy:", test_acc)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7966 - loss: 0.6780 
test accuracy: 0.7594936490058899


## individual test

In [177]:
d = screen.get_time_mac(758990000)
a, b, c = d[1]/12, d[2]/24, d[5]/6
print(str(a) + " " + str(b) + " " + str(c))

predict = model.predict(np.array([[a, b, c]]))
index = np.argmax(predict)
print(apps[index])

0.08333333333333333 0.375 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
off
